In [ ]:
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

In [4]:
os.environ['HF_HOME'] = "W:/huggingface_cache"

In [ ]:
embeddingmodel = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B", model_kwargs={"trust_remote_code": True}, encode_kwargs={"normalize_embeddings":True})
result = embeddingmodel.embed_query("Hi! This is mohan")
print(result)

1024


In [7]:
from langchain.schema import Document
doc1 = Document(page_content="Virat Kohli is one of the most successful and consistent batsman in IPL History, known for his aggresive batting style", metadata={"team":"Royal Challengers Banglore"})

doc2 = Document(page_content="Rohit Sharma is one of the most successful captain in IPL History, leading mumbai Indians to five titles. he is known for his captaincy", metadata={"team":"Mumbai Indians"})

doc3 = Document(page_content="Ms Dhoni, Famously known as Captain cool, has led chennai super kings to multiple IPL Titles. His finishing skills, Wicket keeping skills are known", metadata={"team":"Chennai Super Kings"})

doc4 = Document(page_content="Jasprit Bumrah is considered one of the fastest bowler in T20 crciekt. Playing for Mumabi Indinas, he is known for his master of precision and pace,unorthodox action and relentless discipline", metadata={"team":"Mumbai Indians"})

doc5 = Document(page_content="Ravindra Jadeja is a Dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super kings, he is known for  sword-wielding", metadata={"team":"Chennai Super Kings"})

docs = [doc1, doc2, doc3, doc4, doc5]

In [10]:
vector_store = Chroma(embedding_function=embeddingmodel, persist_directory="9.VectorStores/vectorDBs", collection_name="sample")
vector_store.add_documents(docs)

['62c3c9eb-789f-4596-8d96-cc7ff2725c42',
 'a28dc09f-f645-46a3-8aef-e6eb48a6fc02',
 '0c111c31-2bc5-4338-82ba-54ce78c4c876',
 '570253b3-144c-4084-ab41-4fc4cfbe06db',
 'e0bebd1a-1b93-4338-9a4f-e10943ad0e80']

In [12]:
# View Documents

vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['62c3c9eb-789f-4596-8d96-cc7ff2725c42',
  'a28dc09f-f645-46a3-8aef-e6eb48a6fc02',
  '0c111c31-2bc5-4338-82ba-54ce78c4c876',
  '570253b3-144c-4084-ab41-4fc4cfbe06db',
  'e0bebd1a-1b93-4338-9a4f-e10943ad0e80'],
 'embeddings': array([[-0.01123478, -0.0328014 , -0.01304645, ..., -0.01747213,
          0.03137603, -0.05412785],
        [-0.01248463, -0.02590124, -0.01501196, ..., -0.02798415,
          0.03158053, -0.05688735],
        [ 0.05272389, -0.06671432, -0.01550192, ..., -0.00392435,
          0.03517723, -0.06936487],
        [-0.04059955, -0.07805686, -0.01309619, ..., -0.00834804,
          0.02966242, -0.03385897],
        [ 0.03545947, -0.02779589, -0.0111889 , ..., -0.02837375,
         -0.01060435, -0.04368109]], shape=(5, 1024)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsman in IPL History, known for his aggresive batting style',
  'Rohit Sharma is one of the most successful captain in IPL History, leading mumbai Indians to five t

In [20]:
# Search Documents

vector_store.similarity_search(query="Who among these are captiains?", k=2)

[Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ms Dhoni, Famously known as Captain cool, has led chennai super kings to multiple IPL Titles. His finishing skills, Wicket keeping skills are known'),
 Document(metadata={'team': 'Mumbai Indians'}, page_content='Rohit Sharma is one of the most successful captain in IPL History, leading mumbai Indians to five titles. he is known for his captaincy')]

In [21]:
# Search with Similarity scores

vector_store.similarity_search_with_score(query="Who among these are captains?", k=2)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Rohit Sharma is one of the most successful captain in IPL History, leading mumbai Indians to five titles. he is known for his captaincy'),
  0.9507157802581787),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ms Dhoni, Famously known as Captain cool, has led chennai super kings to multiple IPL Titles. His finishing skills, Wicket keeping skills are known'),
  0.9565247297286987)]

In [22]:
# meta-data filtering
vector_store.similarity_search_with_score(query="", filter={"team": "Chennai Super Kings"})

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ms Dhoni, Famously known as Captain cool, has led chennai super kings to multiple IPL Titles. His finishing skills, Wicket keeping skills are known'),
  1.4767405986785889),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a Dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super kings, he is known for  sword-wielding'),
  1.5706638097763062)]

In [24]:
# Update Documents

updated_doc1 = Document(page_content="Virat Kohli, The former captain of Royal Challengers Banglore (RCB), is renowed for his aggresive leadership and consistency", metadata={"team":"Royal Challengers Banglore"})

vector_store.update_document(document_id="62c3c9eb-789f-4596-8d96-cc7ff2725c42", document=updated_doc1)

In [25]:
vector_store.get(include=["embeddings", "documents", "metadatas"])

{'ids': ['62c3c9eb-789f-4596-8d96-cc7ff2725c42',
  'a28dc09f-f645-46a3-8aef-e6eb48a6fc02',
  '0c111c31-2bc5-4338-82ba-54ce78c4c876',
  '570253b3-144c-4084-ab41-4fc4cfbe06db',
  'e0bebd1a-1b93-4338-9a4f-e10943ad0e80'],
 'embeddings': array([[ 0.0066123 , -0.03016725, -0.01561238, ..., -0.02056137,
          0.01098124, -0.06857778],
        [-0.01248463, -0.02590124, -0.01501196, ..., -0.02798415,
          0.03158053, -0.05688735],
        [ 0.05272389, -0.06671432, -0.01550192, ..., -0.00392435,
          0.03517723, -0.06936487],
        [-0.04059955, -0.07805686, -0.01309619, ..., -0.00834804,
          0.02966242, -0.03385897],
        [ 0.03545947, -0.02779589, -0.0111889 , ..., -0.02837375,
         -0.01060435, -0.04368109]], shape=(5, 1024)),
 'documents': ['Virat Kohli, The former captain of Royal Challengers Banglore (RCB), is renowed for his aggresive leadership and consistency',
  'Rohit Sharma is one of the most successful captain in IPL History, leading mumbai Indians to 

In [26]:
# Delete Document
vector_store.delete(ids=["e0bebd1a-1b93-4338-9a4f-e10943ad0e80"])

In [28]:
# View Documents
vector_store.get(include=["embeddings","documents", "metadatas"])

{'ids': ['62c3c9eb-789f-4596-8d96-cc7ff2725c42',
  'a28dc09f-f645-46a3-8aef-e6eb48a6fc02',
  '0c111c31-2bc5-4338-82ba-54ce78c4c876',
  '570253b3-144c-4084-ab41-4fc4cfbe06db'],
 'embeddings': array([[ 0.0066123 , -0.03016725, -0.01561238, ..., -0.02056137,
          0.01098124, -0.06857778],
        [-0.01248463, -0.02590124, -0.01501196, ..., -0.02798415,
          0.03158053, -0.05688735],
        [ 0.05272389, -0.06671432, -0.01550192, ..., -0.00392435,
          0.03517723, -0.06936487],
        [-0.04059955, -0.07805686, -0.01309619, ..., -0.00834804,
          0.02966242, -0.03385897]], shape=(4, 1024)),
 'documents': ['Virat Kohli, The former captain of Royal Challengers Banglore (RCB), is renowed for his aggresive leadership and consistency',
  'Rohit Sharma is one of the most successful captain in IPL History, leading mumbai Indians to five titles. he is known for his captaincy',
  'Ms Dhoni, Famously known as Captain cool, has led chennai super kings to multiple IPL Titles. Hi